In [ ]:
!pip3 install -qU tensorflow_hub sacremoses --upgrade

In [ ]:
from sacremoses import MosesTokenizer

import tensorflow as tf
import tensorflow_hub as hub

# Needed because I use TF 2.0
tf.compat.v1.disable_eager_execution()

# Grab from TF hub
#elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)

# Or download and extract 
# wget "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" -O elmo.tar.gz
# tar xvfz elmo.tar.gz -C elmo

elmo = hub.Module("elmo/", trainable=False)

In [ ]:
elmo_intro = """
Extensive experiments demonstrate that ELMo representations work extremely well in practice.
We first show that they can be easily added to existing models for six diverse and challenging language understanding problems, including textual entailment, question answering and sentiment analysis.
The addition of ELMo representations alone significantly improves the state of the art in every case, including up to 20% relative error reductions.
For tasks where direct comparisons are possible, ELMo outperforms CoVe (McCann et al, 2017), which computes contextualized representations using a neural machine translation encoder.
Finally, an analysis of both ELMo and CoVe reveals that deep representations outperform those derived from just the top layer of an LSTM.
Our trained models and code are publicly available, and we expect that ELMo will provide similar gains for many other NLP problems.
"""

In [ ]:
def preprocess(text):
    mt = MosesTokenizer(lang='en')
    tokens = mt.tokenize(text, return_str=True)               # tokenize text
    tokens = tokens.split('.')                                # --> array of sentences
    
    tokens = [ x.split(' ') for x in tokens]                  # --> array of arrays of tokens
    tokens = [ [y for y in x if y!=''] for x in tokens]       # remove any empty token
    tokens = [ x for x in tokens if x!=[] ]                   # remove any empty array
    
    tokens_length = [len(x) for x in tokens]                  # compute array lengths
    max_len = max(tokens_length)                              # find longest array
    tokens = [ x + [''] * (max_len - len(x)) for x in tokens] # pad other arrays
    tokens_length = [len(x) for x in tokens]                  # compute array lengths again

    return tokens, tokens_length

In [ ]:
tokens, tokens_length = preprocess(elmo_intro)

In [ ]:
for (x,y) in zip(tokens,tokens_length):
    print(x)
    print(y)

In [ ]:
def do_elmo(tokens, tokens_length):
    embeddings = elmo(
        inputs={
            "tokens": tokens,
            "sequence_len": tokens_length
        },
        signature="tokens",
        as_dict=True)["elmo"]
    
    # tf.compat.v1 is needed because I use TF 2.0
    with tf.compat.v1.Session() as session:
        session.run([tf.compat.v1.global_variables_initializer(), 
                     tf.compat.v1.tables_initializer()])
        elmo_embeddings = session.run(embeddings)
    return elmo_embeddings

In [ ]:
elmo_embeddings = do_elmo(tokens, tokens_length)

In [ ]:
# Print embeddings for the first word of the first sentence
print(elmo_embeddings[0][0]) 

OK, now...

In [ ]:
silly_text = """
Kevin, stop throwing rocks at my car. 
This is totally stupid. 
Now, lets talk about more interesting things. 
Did you know that machine learning rocks? 
It is totally awesome!
"""

In [ ]:
tokens, tokens_length = preprocess(silly_text)

In [ ]:
elmo_embeddings = do_elmo(tokens, tokens_length)

In [ ]:
# 'rocks' in the first sentence : [0][3]
rocks1 = elmo_embeddings[0][3]
# 'rocks' in the fourth sentence : [3][6]
rocks2 = elmo_embeddings[3][6]

print(rocks1) 
print(rocks2) 

Let's compute the cosine similarity of these two vectors.

In [ ]:
import numpy as np
np.dot(rocks1, rocks2) / (np.linalg.norm(rocks1)*np.linalg.norm(rocks2)) 

Not the same!